In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Attention, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

2024-08-27 15:47:47.775604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 15:47:47.792457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 15:47:47.797495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 15:47:47.810462: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-27 15:47:48.557598: W tensorflow/comp

In [2]:
latent_dim = 256
num_samples = 10000

In [5]:
# Step 1: Load and preprocess the data
# Load dataset (replace with actual path)
data_path = 'dataset/eng_to_hindi.xlsx'  # Replace with actual path
df= pd.read_excel(data_path)

In [4]:
# Randomly sample 10,000 rows
#df = DF.sample(n=10000)

# Select the first 10,000 rows
#df = DF.head(15000)

# Select the last 10,000 rows
#df = DF.tail(15000)

In [6]:
english_sentences = df['English words/sentences'].tolist()
hindi_sentences = df['Unnamed: 1'].tolist()


In [7]:
english_sentences

['Hi.',
 'Run!',
 'Run!',
 'Who?',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Get up.',
 'Go now.',
 'Go now.',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'I fell.',
 'I fell.',
 'I know.',
 'I left.',
 'I left.',
 'I lied.',
 'I lost.',
 'I paid.',
 "I'm 19.",
 "I'm OK.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'Really?',
 'Really?',
 'Really?',
 'Thanks.',
 'We try.',
 'We won.',
 'We won.',
 'We won.',
 'We won.',
 'Ask Tom.',
 'Awesome!',
 'Be calm.',
 'Be calm.',
 'Be calm.',
 'Be cool.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be kind.'

In [8]:
# Preprocess data

# Add start and end tokens to Hindi sentences
hindi_sentences = ['\t ' + sentence + ' \n' for sentence in hindi_sentences]

# Tokenization and sequence padding
num_words = 10000  # Maximum number of words to keep in tokenizer
max_sequence_length = 50  # Maximum sequence length for padding

In [9]:
hindi_sentences

['\t नमस्ते। \n',
 '\t दौड़ना! \n',
 '\t दौड़ना! \n',
 '\t कौन? \n',
 '\t बहुत खूब! \n',
 '\t आग! \n',
 '\t मदद करना! \n',
 '\t कूदना। \n',
 '\t रुकना! \n',
 '\t रुकना! \n',
 '\t रुकना! \n',
 '\t इंतज़ार! \n',
 '\t इंतज़ार! \n',
 '\t जारी रखें। \n',
 '\t जारी रखें। \n',
 '\t जारी रखें। \n',
 '\t नमस्ते! \n',
 '\t नमस्ते! \n',
 '\t अच्छा ऐसा है। \n',
 '\t मैं कोशिश करता हूँ। \n',
 '\t मैं जीत गया! \n',
 '\t मैं जीत गया! \n',
 '\t मैं जीत गया. \n',
 '\t अरे नहीं! \n',
 '\t आक्रमण करना! \n',
 '\t आक्रमण करना! \n',
 '\t प्रोत्साहित करना! \n',
 '\t प्रोत्साहित करना! \n',
 '\t प्रोत्साहित करना! \n',
 '\t प्रोत्साहित करना! \n',
 '\t उठना। \n',
 '\t अब जाओ। \n',
 '\t अब जाओ। \n',
 '\t अब जाओ। \n',
 '\t समझ गया! \n',
 '\t समझ गया! \n',
 '\t समझ गया? \n',
 '\t समझ गया? \n',
 '\t समझ गया? \n',
 '\t अंदर कूदो. \n',
 '\t अंदर कूदो. \n',
 '\t मुझे गले लगाओ। \n',
 '\t मुझे गले लगाओ। \n',
 '\t मैं गिर गया। \n',
 '\t मैं गिर गया। \n',
 '\t मुझे पता है। \n',
 '\t मैंने। \n',
 '\t मैंने। \n',
 '\t मैंने 

In [10]:
# English tokenizer
english_tokenizer = Tokenizer(num_words=num_words, filters=' ')
english_tokenizer.fit_on_texts(english_sentences)
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)
english_sequences_padded = pad_sequences(english_sequences, maxlen=max_sequence_length, padding='post')

In [11]:
print(english_tokenizer.word_index)

{'i': 1, 'you': 2, 'to': 3, 'the': 4, 'a': 5, 'is': 6, 'tom': 7, 'he': 8, 'of': 9, 'do': 10, 'in': 11, 'that': 12, "don't": 13, 'have': 14, 'was': 15, 'this': 16, 'my': 17, 'for': 18, "i'm": 19, 'it': 20, 'your': 21, 'are': 22, 'what': 23, 'we': 24, 'be': 25, 'me': 26, 'want': 27, 'not': 28, 'she': 29, 'like': 30, 'know': 31, 'with': 32, 'on': 33, 'you.': 34, 'can': 35, "you're": 36, 'his': 37, 'at': 38, 'did': 39, 'how': 40, 'all': 41, 'they': 42, 'think': 43, 'and': 44, "it's": 45, "can't": 46, 'go': 47, 'very': 48, "didn't": 49, 'about': 50, 'get': 51, 'were': 52, 'as': 53, 'no': 54, 'will': 55, 'me.': 56, 'if': 57, 'had': 58, 'just': 59, 'why': 60, 'that.': 61, 'has': 62, 'it.': 63, 'going': 64, 'him': 65, 'one': 66, 'would': 67, 'there': 68, 'her': 69, 'need': 70, "i'll": 71, 'so': 72, 'tell': 73, 'an': 74, 'when': 75, 'see': 76, 'who': 77, 'really': 78, 'out': 79, 'good': 80, "that's": 81, "i've": 82, 'by': 83, 'should': 84, 'from': 85, 'time': 86, 'never': 87, 'could': 88, 'been

In [12]:
# Hindi tokenizer
hindi_tokenizer = Tokenizer(num_words=num_words, filters='')
hindi_tokenizer.fit_on_texts(hindi_sentences)
hindi_sequences = hindi_tokenizer.texts_to_sequences(hindi_sentences)
hindi_sequences_padded = pad_sequences(hindi_sequences, maxlen=max_sequence_length, padding='post')

In [13]:
hindi_sequences_padded.shape

(175621, 50)

In [14]:
hindi_sequences_padded[0]

array([   1, 8954,    2,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [15]:
# Get the total number of unique tokens
num_encoder_tokens = len(english_tokenizer.word_index) + 1
num_decoder_tokens = len(hindi_tokenizer.word_index) + 1

# Define input and target data
encoder_input_data = english_sequences_padded
decoder_input_data = hindi_sequences_padded[:, :-1]
decoder_target_data = hindi_sequences_padded[:, 1:]

# Split data into training and validation sets
encoder_input_train, encoder_input_val, decoder_input_train, decoder_input_val, decoder_target_train, decoder_target_val = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.2
)

In [16]:
# Step 2: Define the model with bidirectional LSTM and attention

# Encoder
latent_dim = 256
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder_bi_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bi_lstm(encoder_embedding)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim * 2)(decoder_inputs)
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Attention mechanism
attention = Attention()
context_vector = attention([decoder_lstm_outputs, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_lstm_outputs])

# Dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2024-08-27 15:49:13.799351: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9454 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [17]:
# Step 3: Train the model
batch_size = 64
epochs = 50

model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_target_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([encoder_input_val, decoder_input_val], decoder_target_val)
)

Epoch 1/50


2024-08-27 15:49:24.132853: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


2196/2196 ━━━━━━━━━━━━━━━━━━━━ 342s 154ms/step - accuracy: 0.8535 - loss: 1.0750 - val_accuracy: 0.8706 - val_loss: 0.8023
Epoch 2/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 333s 152ms/step - accuracy: 0.8788 - loss: 0.7579 - val_accuracy: 0.8890 - val_loss: 0.6772
Epoch 3/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 333s 152ms/step - accuracy: 0.8930 - loss: 0.6453 - val_accuracy: 0.8950 - val_loss: 0.6156
Epoch 4/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 336s 153ms/step - accuracy: 0.9027 - loss: 0.5683 - val_accuracy: 0.9063 - val_loss: 0.5396
Epoch 5/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 338s 154ms/step - accuracy: 0.9108 - loss: 0.5059 - val_accuracy: 0.9124 - val_loss: 0.4924
Epoch 6/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 339s 154ms/step - accuracy: 0.9173 - loss: 0.4573 - val_accuracy: 0.9174 - val_loss: 0.4568
Epoch 7/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 337s 154ms/step - accuracy: 0.9232 - loss: 0.4158 - val_accuracy: 0.9222 - val_loss: 0.4216
Epoch 8/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 339s 154ms/step - accuracy: 0.9

In [18]:
# Step 4: Define inference models for prediction

# Encoder model for inference
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

# Decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim * 2,))
decoder_state_input_c = Input(shape=(latent_dim * 2,))
decoder_hidden_state_input = Input(shape=(None, latent_dim * 2))

decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=[decoder_state_input_h, decoder_state_input_c]
)

context_vector = attention([decoder_lstm_outputs, decoder_hidden_state_input])
decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_lstm_outputs])
decoder_outputs = decoder_dense(decoder_combined_context)

decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs] + [state_h, state_c]
)

In [19]:
print(hindi_tokenizer.word_index)

{'\t': 1, '\n': 2, 'नहीं': 3, 'मैं': 4, 'मुझे': 5, 'कि': 6, 'आप': 7, 'क्या': 8, 'है.': 9, 'में': 10, 'है।': 11, 'के': 12, 'से': 13, 'वह': 14, 'टॉम': 15, 'एक': 16, 'यह': 17, 'की': 18, 'है': 19, 'को': 20, 'कर': 21, 'पर': 22, 'का': 23, 'लिए': 24, 'है?': 25, 'मैंने': 26, 'बहुत': 27, 'हैं।': 28, 'हैं.': 29, 'हैं?': 30, 'आपको': 31, 'कुछ': 32, 'रहा': 33, 'हो': 34, 'ने': 35, 'हम': 36, 'कोई': 37, 'मेरे': 38, 'भी': 39, 'करना': 40, 'करने': 41, 'तुम्हें': 42, 'ऐसा': 43, 'और': 44, 'तुम': 45, 'था।': 46, 'रहे': 47, 'अपने': 48, 'पास': 49, 'लगता': 50, 'तो': 51, 'हूं.': 52, 'चाहता': 53, 'ही': 54, 'उसने': 55, 'आपके': 56, 'इस': 57, 'उसे': 58, 'था.': 59, 'समय': 60, 'बात': 61, 'हूं।': 62, 'सकता': 63, 'इसे': 64, 'पसंद': 65, 'वे': 66, 'मेरी': 67, 'सकते': 68, 'साथ': 69, 'जो': 70, 'अपनी': 71, 'आपने': 72, 'काम': 73, 'हूं': 74, 'हैं': 75, 'आपकी': 76, 'तक': 77, 'हमें': 78, 'गया': 79, 'बारे': 80, 'किसी': 81, 'उन्होंने': 82, 'क्यों': 83, 'पता': 84, 'अब': 85, 'मेरा': 86, 'था': 87, 'घर': 88, 'अपना': 89, 'कभी': 90, 'कर

In [26]:
# Step 5: Function for decoding sequences
def decode_sequence(input_seq):
    # Encode the input as state vectors
    encoder_output, h, c = encoder_model.predict(input_seq,verbose=0)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = hindi_tokenizer.word_index['\t']

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [encoder_output, h, c],verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = hindi_tokenizer.index_word[sampled_token_index]
        decoded_sentence += sampled_char

        # Add a space if the current character is not a space and the next character is not a punctuation
        if sampled_char not in (' ', '\n') and len(decoded_sentence) > 1 and not decoded_sentence[-2] in (' ', '\n'):
            decoded_sentence += ' '  
        # Exit condition: either hit max length or find stop character
        if (sampled_char == '\n' or len(decoded_sentence) > max_sequence_length):
            stop_condition = True

        # Update the target sequence (length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        h, c = [h, c]

    return decoded_sentence

In [ ]:
 'I spoke slowly so that they could understand me.',
 'I started to learn French in junior high school.',
 "I still haven't gotten over what happened to me.",
 'I suggested that we bring the meeting to an end.',
 'I suggested that we bring the meeting to an end.',
 "I suppose you're already packed and ready to go.",
 "I suppose you're already packed and ready to go.",
 "I suppose you're already packed and ready to go.",
 "I suppose you're already packed and ready to go.",
 "I suppose you're already packed and ready to go.",

In [27]:
# Test the model
test_sentence = "I spoke slowly so that they could understand me."
test_sentence_seq = english_tokenizer.texts_to_sequences([test_sentence])
test_sentence_padded = pad_sequences(test_sentence_seq, maxlen=max_sequence_length, padding='post')
decoded_sentence = decode_sequence(test_sentence_padded)
print('Decoded Sentence:', decoded_sentence)

Decoded Sentence: मैं धीरे-धीरे बोला ताकि वे मेरी बात समझ सकें. 

